<a href="https://colab.research.google.com/github/RaihanAzharRafi/Machine-Learning/blob/main/WEEK%2010/Week_10_PyTorch_buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BUILD THE NEURAL NETWORK**
### **Nama : Raihan Azhar Rafi**
### **Kelas : TK-42-02**
### **Nim : 1103180166**

Sumber Saya Peroleh Dari : https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ ||
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_.
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we
continue to use the CPU.



In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_.
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2977, -0.3111,  0.2327,  0.0387,  0.1887,  0.0837,  0.2833, -0.0934,
         -0.2784,  0.0460, -0.1023, -0.0285, -0.0328, -0.5062, -0.0825, -0.2005,
          0.0037, -1.0463,  0.0645,  0.1941],
        [ 0.5400, -0.0398, -0.0523, -0.4824,  0.2804,  0.1857, -0.0439,  0.0842,
         -0.2528,  0.0125, -0.1546, -0.4572,  0.1232, -0.0991, -0.3078, -0.2631,
         -0.0482, -0.8679, -0.0951, -0.1866],
        [ 0.2691, -0.0505,  0.1971, -0.2460,  0.0962, -0.1875, -0.3623,  0.0964,
         -0.4453, -0.0260,  0.0067, -0.1127, -0.0528, -0.4835,  0.2151,  0.2204,
          0.1011, -0.8652, -0.4077, -0.2854]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2977, 0.0000, 0.2327, 0.0387, 0.1887, 0.0837, 0.2833, 0.0000, 0.0000,
         0.0460, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0037, 0.0000,
         0.0645, 0.1941],
        [0.5400, 0.0000, 0.0000, 0.0000, 0.2804, 0.1857, 0.0000, 0.0842, 0.0000,
         0.0125, 0.0000, 0.0000, 0.1232, 0.0000, 0.00

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0322, -0.0154,  0.0172,  ...,  0.0196, -0.0352,  0.0307],
        [-0.0180,  0.0029,  0.0314,  ...,  0.0261, -0.0314,  0.0173]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0348, -0.0068], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0245,  0.0201,  0.0354,  ...,  0.0191,  0.0348,  0.0183],
        [-0.0326, -0.0196, -0.0161,  ...,  0.0178,  0.0007,  0.0394]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 